# <center>MDI343 - TP Text Mining - Alexandre Durand - 2017/2018</center>  
# <center>Application à la classification : l’Analyse d’opinions</center>

In [1]:
# Authors: Alexandre Gramfort
#          Chloe Clavel
# License: BSD Style.
# TP Cours ML Telecom ParisTech MDI343

import os.path as op
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_val_score
from time import time


# Load data
print("Loading dataset")

from glob import glob
filenames_neg = sorted(glob(op.join('.', 'data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('.', 'data', 'imdb1', 'pos', '*.txt')))

texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]
texts = texts_neg + texts_pos
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0

print("%d documents" % len(texts))

Loading dataset
2000 documents


# Implémentation du classifieur
***

### Questions 1:

#### Compléter la fonction $count\_words$ qui va compter le nombre d’occurrences de chaque mot dans une liste de string et renvoyer le vocabulaire.

In [2]:
def count_words(texts):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts

    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
        n_samples == number of documents.
        n_features == number of words in vocabulary.
    """
    # Construction d'un ensemble (un 'set') du vocabulaire
    words = set()
    for text in texts:
        for w in text.split():
            words.add(w.strip())

    # Construction d'un dictionnaire {word : index of the word}
    n_features = len(words)
    dict_w_idx = dict(zip(words, range(n_features)))

    # Creation et Remplissage de la matrice "counts" (généralement appelée: X)
    counts = np.zeros((len(texts), n_features))

    for i, text in enumerate(texts):
        for w in text.split():
            j = dict_w_idx[w.strip()]
            counts[i, j] = counts[i, j] + 1

    return dict_w_idx, counts

In [3]:
# Test de la fonction "count_words" sur le corpus entier
start = time()

vocabulary, X = count_words(texts)

print(" (count_words : execution time = %0.2f s)" % (time() - start))
print("Taille du Vocabulaire =", len(vocabulary))
print("Taille de la matrice X =", X.shape)

 (count_words : execution time = 1.87 s)
Taille du Vocabulaire = 50920
Taille de la matrice X = (2000, 50920)


### Question 2.
#### Expliquer comment les classes positives et négatives ont été assignées sur les critiques de films (voir fichier poldata.README.2.0)

> Pour déterminer si une critique était positive ou négative, comme aucune notation n'était accessible via un format structuré, les auteurs du fichier ont recherché des indications dans le texte. Cela pouvait être, entre autres, des notations numériques du type (8/10) ou via un système d'étoiles (3 étoiles sur 5).

> Les règles suivantes ont ensuite été appliquées :  
- Sur une échelle de $0$ à $5$ :
    - Note $\ \geqslant \ 3.5 \ / \ 5  \ \ \Longrightarrow$ critique positive
    - Note $\ \leqslant \ \ \ \ 2 \ / \ 5 \ \ \Longrightarrow$ critique négative  

>  
- Sur une échelle de $0$ à $4$ :
    - Note $\ \geqslant \ \ \ \ 3 \ / \ 4 \ \ \Longrightarrow$ critique positive
    - Note $\ \leqslant \ 1.5 \ / \ 5  \ \ \Longrightarrow$ critique negative  
    
>  
- Avec un système de lettres :
    - Note $\ \geqslant \ B \ \ \ \ \Longrightarrow$ critique positive
    - Note $\ \leqslant \ C^- \ \Longrightarrow$ critique negative

### Question 3.
#### Compléter la classe $NB$ pour qu’elle implémente le classifieur Naive Bayes en vous appuyant sur le pseudo-code de la figure 1 et sa documentation.  
Explications pour la méthode $fit$ :
> Le choix a été fait d'intégrer la fonction $count\_words$ à l'intérieur de la méthode $fit$.
> Ainsi le $fit$ de notre classe $NB$ prendra directement en arguments le corpus de textes d'apprentissage ainsi que les labels associés. 

> En sortie, on obtiendra :
- $C$, l'ensemble des différentes classes
- $Vocabulary\_train$, le Vocabulaire du corpus d'entrainement
- $prior$, la matrice des probabilités "à priori"
- $condProb$, la matrice des probabilités conditionnelles d'apparation des termes en fonction de la classe

Explications pour la méthode $predict$ :
> Là encore, on donnera un texte (ou un corpus de textes) à prédire directement en entrée de la méthode.  
Un $count\_words$ sur ces textes nous permettra de récupèrerer le "vocabulaire_test" associé. Après intersection des deux vocabulaires "Train" et "Test", on réduira les matrices $condProb$ (issu de la méthode $fit$) et $X\_test$ aux termes communs afin de pouvoir faire le produit matriciel ci-dessous, nécessaire au calcul des scores de chaque classe : $$scores = \mathit{log} \ prior + X\_test \cdot (\mathit{log} \ condProb)^{T}$$  

> En sortie, on obtiendra :
- $y\_pred$, le vecteur des prédictions des textes données en entrée  

> L'avantage de cette implémentation réside dans le fait qu'on n'est pas contraint de fournir des matrices $X\_train$ et $X\_test$ ayant le même nombre de colonnes (c'est-à-dire issu du même Vocubulaire).

Explications pour la méthode $score$ :
> Pour un corpus de textes à prédire et un vecteur contenant les vrais labels des textes, la méthode $score$ renvoie un score de prédiction qui est le rapport entre le nombre de textes qui ont été bien prédits et le nombre total de textes qu'il fallait prédire.

In [4]:
class NB(BaseEstimator, ClassifierMixin):
    
    def __init__(self):
        pass

    def fit(self, texts_train, y_train):

        # Etape préliminaire de Count_Word
        vocabulary_train, X_train = count_words(texts_train)

        V = X_train.shape[1]  # taille du Vocabulaire Train
        N = X_train.shape[0]  # Nbre total de Documents (= len(y_train))
        C = np.unique(y_train)  # Ensemble des Classes

        # Initialisation des matrices de calcul
        prior = np.zeros(len(C))
        T_ct = np.zeros((len(C), V))
        condProb = np.zeros((len(C), V))

        # Remplissage des matrices, classe par classe
        for idx, c in enumerate(C):
            # Probabilités "à priori"
            Nc = len(y_train[y_train == c]) / N
            prior[idx] = Nc

            # Matrice T_ct (occurrence totale par classe, pour chacun des termes)
            T_ct[idx, :] = np.sum(X_train[y_train == c, :], axis=0)

            # Matrice cond_prob avec Lissage de Laplace
            alpha = 1
            T_ct_total = np.sum(T_ct[idx, :] + alpha)
            condProb[idx, :] = (T_ct[idx, :] + alpha) / T_ct_total

        self.C = C
        self.vocabulary_train = vocabulary_train
        self.prior = prior
        self.condProb = condProb
        return self

    def predict(self, texts_test):

        # Count_Words & initialisation
        self.vocabulary_test, X_test = count_words(texts_test)
        y_pred = np.zeros(X_test.shape[0])

        # Intersection des deux "Vocabulary" Train & Test
        shared_terms = self.vocabulary_train.keys() & self.vocabulary_test.keys()

        # Pour chacun des 2 dictionnaires (train & test), trouver les indices des termes communs
        idx_toKeep_train = []
        idx_toKeep_test = []
        for t in shared_terms:
            idx_toKeep_train.append(self.vocabulary_train[t])
            idx_toKeep_test.append(self.vocabulary_test[t])

        # Réduction de la matrice condProb aux termes communs
        reduced_condProb = self.condProb[:, idx_toKeep_train]

        # Réduction de la matrice X_test aux termes communs
        reduced_X_test = X_test[:, idx_toKeep_test]

        # Calcul des scores = log prior + reduced_X_test.(log reduced_condProb)^T
        log_reduced_condProb = np.log(reduced_condProb)
        log_prior = np.log(self.prior)
        scores = log_prior + np.dot(reduced_X_test, log_reduced_condProb.T)

        # Prediction = classe qui a le plus grand score
        maxScore = np.argmax(scores, axis=1)
        y_pred = self.C[maxScore]

        return y_pred

    def score(self, texts_test, y_test):
        return np.mean(self.predict(texts_test) == y_test)

### Question 4.
#### Evaluer les performances de votre classifieur en cross-validation 5-folds.

In [5]:
cv_scores = cross_val_score(NB(), texts, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.815 (+/- 0.027)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    8.4s finished


### Question 5.
#### Modifiez la fonction $count\_words$ pour qu’elle ignore les “stop words” dans le fichier *data/english.stop*. Les performances sont-elles améliorées ?

Explications :
> Comme nous utilisons la fonction $count\_words$ dans la classe $NB$, plutôt que de modifier la fonction $count\_words$ elle-même (ce qui impliquerait de recréer une classe $NB$ avec cette nouvelle fonction), nous allons plutôt retirer les "stop words" directement dans le corpus de textes.  

> C'est ce que fait la fonction $retrieve\_stopW$ ci-dessous, qui prend en argument une liste de textes ainsi qu'un ensemble de "stop words" et retourne une nouvelle liste de ces mêmes textes mais sans les "stop words".

In [6]:
def retrieve_stopW(texts, stopWords):
    texts_cleaned = []
    for text in texts:
        text_cleaned = [w.strip() for w in text.split() if w.strip() not in stopWords]
        texts_cleaned.append(" ".join(text_cleaned))
    return texts_cleaned

In [7]:
# Lecture du fichier de stop_words
english_StopWords = set(open('./data/english.stop').read().split('\n'))

# Nettoyage des "stop words" dans le corpus
texts_cleaned = retrieve_stopW(texts, english_StopWords)

In [8]:
# Comparaison avec le corpus de textes initial (i.e. avec les "stop words")
vocabulary, X = count_words(texts)
vocabulary_cleaned, X_cleaned = count_words(texts_cleaned)

print("'count_words' sur le corpus de textes sans les 'stop words' (par rapport à avec) :")

diff_feat = len(vocabulary) - len(vocabulary_cleaned)
p_diff_feat = diff_feat * 100 / len(vocabulary)
print("==> Nombre de termes uniques (features) en moins =", diff_feat,
      "(soit %0.2f" % p_diff_feat + "%)")

diff_occur = X.sum() - X_cleaned.sum()
p_diff_occur = diff_occur * 100 / X.sum()
print("==> Nombre d'occurrences en moins = %d" % diff_occur,
      "(soit %0.2f" % p_diff_occur + "%)")

'count_words' sur le corpus de textes sans les 'stop words' (par rapport à avec) :
==> Nombre de termes uniques (features) en moins = 545 (soit 1.07%)
==> Nombre d'occurrences en moins = 718127 (soit 48.11%)


In [9]:
# Evaluation des performances sans les 'stop words'
cv_scores = cross_val_score(NB(), texts_cleaned, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.805 (+/- 0.025)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.3s finished


** Conclusion :**
> En retirant les 'stop words', le score est sensiblement identique à celui calculé précédemment sur les textes originaux.

> Ce résultat semble assez logique si on fait l'hypothèse, plutôt réaliste, qu'il y a autant de 'stop words' dans les critiques positives que dans les critiques négatives. La présence ou non de ces mots n'a donc peu d'influence dans le choix du label à prédire.

> Les mots véritablement discrimants sont ceux qui sont présents en plus grande majorité dans les textes d'un label plutôt que d'un autre.

> Par contre, l'avantage de ne pas inclure les 'stop words' se trouve dans la taille réduite des objets à manipuler.
En comparant les résultats d'un $count\_words$ sur le corpus original et sur le corpus dont les 'stop words' ont été retirés, on se rend compte qu'on a retirés pratiquement 50% des occurences, tous termes confondus.  
Si on devait traiter une masse de textes beaucoup plus imposante, cela aurait peut-être une influence siginificative sur les temps de calcul.

# Utilisation de scikitlearn
***

### Questions 1.

#### Comparer votre implémentation avec scikitlearn.

On utilisera la classe CountVectorizer et un Pipeline.

Vous expérimenterez en autorisant les mots et bigrammes ou en travaillant sur les sous-chaines de caractères (option analyzer='char').

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

> Avant d'expérimenter sur les mots et les bigrammes, nous allons utiliser une première fois le classifieur de ScikitLearn en utilisant seulement les mots (comme nous avons fait jusqu'à présent) afin d'avoir un point de comparaison entre les différentes méthodes.

In [11]:
# Travail sur les mots seulement :  analyzer='word' & ngram_range=(1, 1)
# --------------------------------
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))
multiNB = MultinomialNB()
pipeline = Pipeline([("vectorizer", vectorizer), ("multiNB", multiNB)])

# Evaluate the models using crossvalidation
cv_scores = cross_val_score(pipeline, texts, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.812 (+/- 0.026)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.1s finished


**Conclusion :**

> Le classifieur de scikitlearn réalise une performance similaire à notre classifieur (légèrement plus faible même) mais en un temps d'execution plus rapide, surement dû aux optimisations computationnelles propres à SKLearn.  

Nous allons maintenant effectuer la classification en prenant en compte à la fois les mots mais aussi les bigrammes (ensemble formé par un mot + le mot qui le précède).

In [12]:
# Travail sur les mots et les bigrammes :  analyzer='word' & ngram_range=(1, 2)
# --------------------------------------
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
multiNB = MultinomialNB()
pipeline = Pipeline([("vectorizer", vectorizer), ("multiNB", multiNB)])

# Evaluate the models using crossvalidation
cv_scores = cross_val_score(pipeline, texts, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.831 (+/- 0.019)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.0s finished


**Conclusion :**
> L'utilisation des mots et bigrammes a permis une nette amélioration du score (pratiquement 3%).  

> Cela prouve qu'au delà du sens d'un mot, c'est bien des ensembles de mots qui apportent de l'information utile à la classification.

> Voyons si des ensembles de mots plus longs n'apporteraient pas plus d'informations. Nous allons utiliser une "GridSearch" pour cela.

In [13]:
from sklearn.model_selection import GridSearchCV

# Pipeline and Parameters
vectorizer = CountVectorizer(analyzer='word')
multiNB = MultinomialNB()
pipeline = Pipeline([("vectorizer", vectorizer), ("multiNB", multiNB)])

parameters = {'vectorizer__ngram_range':[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6)]}

# Fit the Grid Search
grid = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=0)
grid.fit(texts, y)

# Display Results
mean_scores = grid.cv_results_['mean_test_score']
for i in range(len(mean_scores)):
    print("ngram_range used :", grid.cv_results_['params'][i]['vectorizer__ngram_range'],
          "  -->  score = %0.3f" % mean_scores[i])

print("\nBest Candidate:")
print("   Best Score : %0.3f" % grid.best_score_)
print("   Best Parameters :", grid.best_params_['vectorizer__ngram_range'])

ngram_range used : (1, 1)   -->  score = 0.812
ngram_range used : (1, 2)   -->  score = 0.831
ngram_range used : (1, 3)   -->  score = 0.823
ngram_range used : (1, 4)   -->  score = 0.817
ngram_range used : (1, 5)   -->  score = 0.814
ngram_range used : (1, 6)   -->  score = 0.810

Best Candidate:
   Best Score : 0.831
   Best Parameters : (1, 2)


**Conclusion :**
> Non, en autorisant des combinaisons encore plus complexes que la seule utilisation des mots et bigrammes, nous n'avons pas fait augmenter la performance du classifieur. Ne conserver que les mots et bigrammes semble donc un choix très judicieux.

Testons maintenant en prenant en compte, non plus des mots entiers, mais seulement des sous-chaines de caractères.  
Pour cela, on utilise le paramètre 'analyzer=char'.

In [14]:
from sklearn.model_selection import GridSearchCV

# Pipeline and Parameters
vectorizer = CountVectorizer(analyzer='char')
multiNB = MultinomialNB()
pipeline = Pipeline([("vectorizer", vectorizer), ("multiNB", multiNB)])

parameters = {'vectorizer__ngram_range':[(3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8),
                                         (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8)]}

# Fit the Grid Search
grid = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=0)
grid.fit(texts, y)

# Display Results
mean_scores = grid.cv_results_['mean_test_score']
for i in range(len(mean_scores)):
    print("ngram_range used :", grid.cv_results_['params'][i]['vectorizer__ngram_range'],
          "  -->  score =", mean_scores[i])

print("\nBest Candidate:")
print("   Best Score :", grid.best_score_)
print("   Best Parameters :", grid.best_params_['vectorizer__ngram_range'])

ngram_range used : (3, 3)   -->  score = 0.7645
ngram_range used : (4, 4)   -->  score = 0.8
ngram_range used : (5, 5)   -->  score = 0.8195
ngram_range used : (6, 6)   -->  score = 0.8275
ngram_range used : (7, 7)   -->  score = 0.828
ngram_range used : (8, 8)   -->  score = 0.829
ngram_range used : (1, 3)   -->  score = 0.7475
ngram_range used : (1, 4)   -->  score = 0.7865
ngram_range used : (1, 5)   -->  score = 0.804
ngram_range used : (1, 6)   -->  score = 0.816
ngram_range used : (1, 7)   -->  score = 0.8215
ngram_range used : (1, 8)   -->  score = 0.818

Best Candidate:
   Best Score : 0.829
   Best Parameters : (8, 8)


** Conclusion :**
> Le meilleur score est obtenu en travaillant sur des sous-chaines de 8 caractères. 

> Ce score est assez proche de celui trouvé précédemment (travail sur les mots & bigrammes, score = 0.831). Ce n'est pas illogique, la taille d'un mot ou d'un ensemble de 2 mots doit probablement s'approcher de ce nombre de caractères.

> Bien que très proche, ce score est malgré tout inférieur au meilleur score obtenu en travaillant sur les mots entiers. Nous allons donc poursuivre les tests en ne considérant que les mots & bigrammes.

### Questions 2.

#### Tester un autre algorithme de la librairie scikitlearn (ex : $LinearSVC$, $LogisticRegression$).

In [15]:
from sklearn.svm import LinearSVC

vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
linear_svc = LinearSVC(penalty='l2', loss='squared_hinge')
pipeline = Pipeline([("vectorizer", vectorizer), ("linear_svc", linear_svc)])

# Evaluate the models using crossvalidation
cv_scores = cross_val_score(pipeline, texts, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.850 (+/- 0.039)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   22.0s finished


In [16]:
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
log_regr = LogisticRegression(penalty='l2', solver='liblinear', n_jobs=-1)
pipeline = Pipeline([("vectorizer", vectorizer), ("log_regr", log_regr)])

# Evaluate the models using crossvalidation
cv_scores = cross_val_score(pipeline, texts, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.853 (+/- 0.033)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   19.1s finished


**Conclusion :**
> Nous avons testé deux autres algorithmes : $linearSVC$ et $LogisticRegression$.  

> Les résultats sont meilleures (d'environ 2%) qu'avec notre implémentation ou encore qu'avec $MultinomialNB$ de SKLearn. Cette amélioration se paye cependant par un temps de calcul supérieur.

### Questions 3.

#### Utiliser la librairie NLTK afin de procéder à une racinisation (stemming). Vous utiliserez la classe SnowballStemmer.

> Comme pour le traitement des 'stop words', nous allons effectuer la racinisation sur le corpus de textes entier en amont de la classification.  

> La fonction $stem\_doc$ ci-dessous réalise ce traitement.  

> De plus, nous allons continuer d'utiliser le classifieur $LogisticRegression$ sur les mots et les bigrammes, combinaison ayant fourni les meilleurs résultats jusqu'à présent.

In [17]:
from nltk import SnowballStemmer

def stem_doc(texts, language):
    stemmer = SnowballStemmer(language)
    texts_stem = []
    for text in texts:
        text_stem = [stemmer.stem(w.strip()) for w in text.split()]
        texts_stem.append(" ".join(text_stem))
    return texts_stem

In [18]:
# Stemming of all the texts
start = time()

texts_stem = stem_doc(texts, 'english')

print(" (stemming : execution time = %0.2f s)" % (time() - start))

 (stemming : execution time = 16.90 s)


In [19]:
# Pipeline
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
log_regr = LogisticRegression(penalty='l2', solver='liblinear', n_jobs=-1)
pipeline = Pipeline([("vectorizer", vectorizer), ("log_regr", log_regr)])

# Evaluate the models using crossvalidation
cv_scores = cross_val_score(pipeline, texts_stem, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.859 (+/- 0.027)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.2s finished


**Conclusion :**
> La racinisation améliore encore plus les résultats.  

> En éliminant les suffixes des mots, cette méthode a pour but de regrouper des variantes morphologiques d'un mot dans une racine commune.

> Le fait que les termes provenant d'une même racine (et donc normalement ayant la même signification) soient, après racinisation, considérés comme identiques permet de ne pas "disperser" l'influence de ces termes en différentes variantes de suffixes.

> Cependant, le temps passé a effectuer la racinisation est ici considérable (pratiquement autant que pour effectuer la classification elle-même). Si l'on met en regard le gain de performance (+ 0.6 %), on peut conclure que la racinisation doit être décidée en fonction des gains de performance réels et du contexte d'utilisation.

### Questions 4.

#### Filtrer les mots par catégorie grammaticale (POS : Part Of Speech) et ne garder que les noms, les verbes, les adverbes et les adjectifs pour la classification.

Voici la liste des "Part-of-Speech tags" provenant du projet Penn Treebank, et que nous allons considérer :

| NOUN                          | VERB                                          | ADVERB                      | ADJECTIVE                      |
|:-------------------------------|:-----------------------------------------------|:-----------------------------|:--------------------------------|
| NN --> Noun, singular or mass | VB --> Verb, base form                        | RB --> Adverb               | JJ --> Adjective               |
| NNS --> Noun, plural          | VBD --> Verb, past tense                      | RBR --> Adverb, comparative | JJR --> Adjective, comparative |
| NNP --> Proper noun, singular | VBG --> Verb, gerund or present participle    | RBS --> Adverb, superlative | JJS --> Adjective, superlative |
| NNPS --> Proper noun, plural  | VBN --> Verb, past participle                 |                             |                                |
|                               | VBP --> Verb, non-3rd person singular present |                             |                                |
|                               | VBZ --> Verb, 3rd person singular present     |                             |                                |

In [20]:
nouns = ['NN', 'NNS', 'NNP', 'NNPS']
verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
adverbs = ['RB', 'RBR', 'RBS']
adjectives = ['JJ', 'JJR', 'JJS']

tags_to_keep = set(nouns + verbs + adverbs + adjectives)

In [21]:
from nltk import pos_tag
from nltk import word_tokenize

def keep_only_desired_tags(texts, tags_to_keep):
    texts_tags = []
    for text in texts:
        tagged_words = pos_tag(word_tokenize(text))
        desired_tags_words = [t[0] for t in tagged_words if t[1] in tags_to_keep]
        texts_tags.append(" ".join(desired_tags_words))
    return texts_tags

In [22]:
# Traitement sur le corpus de textes
start = time()

texts_good_tags = keep_only_desired_tags(texts, tags_to_keep)

print(" (POS tag : execution time = %0.2f s)" % (time() - start))

 (POS tag : execution time = 67.57 s)


In [23]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))
log_regr = LogisticRegression(penalty='l2', solver='liblinear', n_jobs=-1)
pipeline = Pipeline([("vectorizer", vectorizer), ("log_regr", log_regr)])

# Evaluate the models using crossvalidation
cv_scores = cross_val_score(pipeline, texts_good_tags, y, cv=5, n_jobs=-1, verbose=1)
print("Accuracy: %0.3f (+/- %0.3f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.852 (+/- 0.026)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   17.1s finished


**Conclusion :**

> Le score n'est pas amélioré en ne gardant que les noms, verbes, adverbes et adjectifs.

## Conclusion générale :

- Le classifieur "Naïve Bayes" que nous avons implémenté possède des performances comparables à celui de ScikitLearn.

- Retirer les 'stop words' n'apporte pas plus de performance car ces mots sont visiblement présents en proportions similaires entre toutes les classes.

- Considérer les mots et les bigrammes est la meilleure option.

- Les autres algorithmes de classification (LinearSVC et LogisticRegression) ont apporté des performances très légèrement supérieures, mais avec des temps de calcul plus longs.

- Considérer comme un unique terme toutes les variantes morphologiques de ce terme (=raciniser) a permis d'accroitre les performances.

- Ne garder dans le corpus que les noms, verbes, adverbes et adjectifs n'a pas permis d'amélioration. De plus, le temps de preprocessing du texte (> 1 min) est rédhibitoire.